In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Thông tin sinh viên

Họ tên: Phan Anh Lộc

MSSV: 19521766

Bài tập thực hành số 3

# Import các thư viện 

In [85]:
import os.path
import cv2 as cv
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from skimage import feature
from sklearn.linear_model import LogisticRegression

# Đường dẫn dataset

In [86]:
dir_monhoc = '/content/drive/MyDrive/CS231.M12.KHCL'
dir_baitap = os.path.join(dir_monhoc, 'Baitap', 'Dataset','dogscats_small')
dir_images_train = dir_baitap + '/train'
dir_images_test = dir_baitap + '/test'

# Tính histogram ảnh xám

In [87]:
def TinhHist(pathfilename):
    img = cv.imread(pathfilename,0)
    hist = cv.calcHist([img],[0],None,[256],[0,256])
    size = img.shape[0]*img.shape[1]
    hist = hist / size
    return hist

# Tính HOG

In [88]:
def compute_hog_train(pathfilename):
    img = cv.imread(pathfilename,0)
    img =cv.resize(img, (256, 256))
    (hog, hog_image) = feature.hog(img, orientations=9,
                      pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                      block_norm='L2-Hys', visualize=True, transform_sqrt=True)
    return hog

# Tính histogram ảnh màu

In [89]:
def TinhHist1(pathfilename):
    img = cv.imread(pathfilename,cv.IMREAD_COLOR)
    img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
    ft = []
    colors = ("r","g","b")
    for i,c in enumerate(colors):
      hist = cv.calcHist([img],[0],None,[256],[0,256])
      ft.extend(hist)
    return ft

# Đọc dữ liệu ảnh (đặc trưng và nhãn của data train)

In [90]:
def ReadData(path_to_files):
    features,features_2,labels = [],[],[]
    for file_name in sorted(os.listdir(path_to_files)):
    #print(file_name)
      label = file_name.split(os.path.sep)[-1].split(".")[0]
      labels.append(label)
      path_filename = os.path.join(path_to_files,file_name)
      hist = TinhHist(path_filename)
      hist1 = TinhHist1(path_filename)
      features.append(hist)
      features_2.append(hist1)
    return features, labels, features_2

In [91]:
features, labels, features_2 = ReadData(dir_images_train)
print(labels)
print(len(labels))

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog']
42


# Đọc dữ liệu ảnh (đặc trưng và nhãn của data test)

In [92]:
test_features,l,test_features_2 = ReadData(dir_images_test)
test_labels = ['dog','cat','cat','dog','cat','cat','cat','cat','dog','dog','cat','dog','cat','dog','dog','cat','cat','cat','cat','cat']
#print(test_features)
len(test_labels)

20

# Sử dụng KNN để phân lớp ảnh (Histogram)

In [93]:
train_features = np.array(features)
train_labels = np.array(labels)

test_features = np.array(test_features)
test_labels = np.array(test_labels)

train_features_shape = train_features.shape
test_features_shape = test_features.shape 

train_features = train_features.reshape(train_features_shape[0],train_features_shape[1])
test_features = test_features.reshape(test_features_shape[0],test_features_shape[1])

train_features_2 = np.array(features_2)
test_features_2 = np.array(test_features_2)

train_features_shape_2 = train_features_2.shape
test_features_shape_2 = test_features_2.shape 

train_features_2 = train_features_2.reshape(train_features_shape_2[0],train_features_shape_2[1])
test_features_2 = test_features_2.reshape(test_features_shape_2[0],test_features_shape_2[1])

Đánh giá độ chính xác qua các tập test (ảnh xám)

In [94]:
model1 = KNeighborsClassifier(n_neighbors=1)
model1.fit(train_features,train_labels)

acc = model1.score(test_features, test_labels)
print("Accuracy (K = 1): ", acc)

Accuracy (K = 1):  0.55


In [95]:
model3 = KNeighborsClassifier(n_neighbors=3)
model3.fit(train_features, train_labels)

acc = model3.score(test_features, test_labels)
print("Accuracy (K = 3): ", acc)

Accuracy (K = 3):  0.45


In [96]:
model5 = KNeighborsClassifier(n_neighbors=5)
model5.fit(train_features, train_labels)

acc = model5.score(test_features, test_labels)
print("Accuracy (K = 5): ", acc)

Accuracy (K = 5):  0.6


Đánh giá độ chính xác qua các tập test (ảnh màu)

In [97]:
modelcolor1 = KNeighborsClassifier(n_neighbors=1)
modelcolor1.fit(train_features_2,train_labels)

acccolor1 = modelcolor1.score(test_features_2, test_labels)
print("Accuracy (K = 1): ", acccolor1)

Accuracy (K = 1):  0.6


In [98]:
modelcolor3 = KNeighborsClassifier(n_neighbors=3)
modelcolor3.fit(train_features_2,train_labels)

acccolor3 = modelcolor3.score(test_features_2, test_labels)
print("Accuracy (K = 3): ", acccolor3)

Accuracy (K = 3):  0.4


In [ ]:
modelcolor5 = KNeighborsClassifier(n_neighbors=5)
modelcolor5.fit(train_features_2,train_labels)

acccolor5 = modelcolor5.score(test_features_2, test_labels)
print("Accuracy (K = 5): ", acccolor5)

Accuracy (K = 5):  0.5


# Sử dụng KNN để phân lớp ảnh (HOG)

Đọc dữ liệu ảnh 

In [99]:
def ReadDataHOG(path_to_files):
    labels = []
    features = []
    for file_name in sorted(os.listdir(path_to_files)):
      # print(file_name)
      label = file_name.split(os.path.sep)[-1].split(".")[0]
      labels.append(label)
      hog = compute_hog_train(os.path.join(path_to_files, file_name))
      features.append(hog)
    return features, labels

In [100]:
train_features_3, train_labels_3 = ReadDataHOG(dir_images_train)

test_features_3, test_labels_3 = ReadDataHOG(dir_images_test)
test_labels_3 = test_labels

train_features_3 = np.array(train_features_3)
train_labels_3 = np.array(train_labels_3)

test_features_3 = np.array(test_features_3)
test_labels_3 = np.array(test_labels_3)

Đánh giá độ chính xác qua các tập test

In [101]:
model1 = KNeighborsClassifier(n_neighbors=1)
model1.fit(train_features_3,train_labels_3)

acc = model1.score(test_features_3, test_labels_3)
print("Accuracy (K = 1): ", acc)

Accuracy (K = 1):  0.35


In [102]:
model3 = KNeighborsClassifier(n_neighbors=3)
model3.fit(train_features_3, train_labels_3)

acc = model3.score(test_features_3, test_labels_3)
print("Accuracy (K = 3): ", acc)

Accuracy (K = 3):  0.4


In [103]:
model5 = KNeighborsClassifier(n_neighbors=5)
model5.fit(train_features_3, train_labels_3)

acc = model5.score(test_features_3, test_labels_3)
print("Accuracy (K = 5): ", acc)

Accuracy (K = 5):  0.4


# Sử dụng Linear Classification để phân lớp ảnh 

ĐỐI VỚI HOG

In [104]:
clf = LogisticRegression(random_state=0).fit(train_features_3, train_labels_3)
acc = clf.score(test_features_3, test_labels_3)
print("Accuracy = ",acc)

Accuracy =  0.65


ĐỐI VỚI HISTOGRAM

In [105]:
clf = LogisticRegression(random_state=0).fit(train_features, train_labels)
acc1 = clf.score(test_features, test_labels)
print("Accuracy: ", acc1)

Accuracy:  0.45
